# Initial exploratory plots

In [31]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import numpy as np

plt.style.use('seaborn-darkgrid')

In [2]:
df = pd.read_csv('DataFrame_16-19_with_dicts.zip', compression = 'zip', low_memory = False)

In [3]:
#df['datetime'] = pd.to_datetime(df['datetime'])
#df = df.set_index('datetime', drop = True)

df.head()

,age_range,datetime,gender,involved_person,legislation,object_of_search,officer_defined_ethnicity,operation,operation_name,outcome,...,removal_of_more_than_outer_clothing,self_defined_ethnicity,type,Force,latitude,street_name,street_id,longitude,outcome_object_id,outcome_object_name
0,18-24,2016-07-01T03:36:40+00:00,Female,True,Misuse of Drugs Act 1971 (section 23),Controlled drugs,White,False,NaN,Suspect summonsed to court,...,False,White - White British (W1),Person search,avon-and-somerset,51.472770,On or near The Avenue,545646.0,-2.633822,summonsed,Suspect summonsed to court
1,18-24,2016-07-01T03:36:40+00:00,Female,True,Misuse of Drugs Act 1971 (section 23),Controlled drugs,White,False,NaN,False,...,False,White - White British (W1),Person search,avon-and-somerset,51.472770,On or near The Avenue,545646.0,-2.633822,nothing,Nothing found - No further action
2,18-24,2016-07-01T03:36:40+00:00,Female,True,Misuse of Drugs Act 1971 (section 23),Controlled drugs,White,False,NaN,False,...,False,White - White British (W1),Person search,avon-and-somerset,51.472770,On or near The Avenue,545646.0,-2.633822,nothing,Nothing found - No further action
3,10-17,2016-07-01T05:20:55+00:00,Male,True,Police and Criminal Evidence Act 1984 (section 1),Articles for use in criminal damage,White,False,NaN,False,...,False,White - White British (W1),Person search,avon-and-somerset,51.013992,On or near Shopping Area,530193.0,-3.101950,nothing,Nothing found - No further action
4,25-34,2016-07-01T07:21:51+00:00,Male,True,Police and Criminal Evidence Act 1984 (section 1),Stolen goods,Other,False,NaN,False,...,True,Not Stated (NS),Person search,avon-and-somerset,51.080631,On or near West Lane,531254.0,-2.612072,nothing,Nothing found - No further action


In [ ]:
df['gender']\
    .value_counts()\
    .plot(kind = 'pie',
          figsize = (6, 6))

In [ ]:
df['object_of_search']\
    .value_counts()\
    .plot(kind='bar',
          figsize = (15, 5))

In [ ]:
df_object = df[df['object_of_search'] != 'Controlled drugs']

df_object['object_of_search']\
    .value_counts()\
    .plot(kind='bar',
          figsize = (15, 5))

In [ ]:
df['removal_of_more_than_outer_clothing'].value_counts().plot(kind='bar')

In [ ]:
df['self_defined_ethnicity']\
    .value_counts()\
    .plot(kind='barh',
          figsize = (15, 10),
          title = 'Self-defined ethnicity')

In [ ]:
df_ethn = df[df['self_defined_ethnicity'] != 'White - English/Welsh/Scottish/Northern Irish/British']

df_ethn['self_defined_ethnicity']\
    .value_counts()\
    .plot(kind='barh',
          figsize = (15, 10),
          title = 'Self-defined ethnicity \n (excluding "White - English/Welsh/Scottish/Northern Irish/British")')

In [ ]:
df['Force']\
    .value_counts()\
    .plot(kind='barh',
          figsize = (15, 10),
          title = 'Police Force')

In [ ]:
force_no_met = df[df['Force'] != 'metropolitan']

force_no_met['Force']\
    .value_counts()\
    .plot(kind='barh',
          figsize = (15, 10),
          title = 'Police Force (exluding the Met)')

In [ ]:
df_gender = df.groupby('gender')

df_gender['object_of_search']\
    .value_counts()\
    .plot(kind='barh',
          figsize = (15, 12),
          title = 'Police Force')

In [ ]:
df_gender = df
pd.get_dummies(df_gender, columns = ['gender'])

In [ ]:
df['outcome'].value_counts()

In [35]:
import folium
from folium import plugins
from folium.plugins import HeatMap

In [36]:
my_map = folium.Map(location=[51.5074, 0.1278],
                    tiles = 'cartodbpositron',
                    zoom_start = 7) # for UK 

my_map

In [37]:
df['latitude'] = df['latitude'].astype(float)
df['longitude'] = df['longitude'].astype(float)
df['failure'] = df.loc[:,'outcome'].isin(['A no further action disposal', 'False']).astype(float)

heat_df = df[['latitude', 'longitude', 'failure']]
heat_df.replace(0.0, np.nan, inplace=True)

heat_df = heat_df.dropna(axis=0, subset=['latitude','longitude', 'failure'])

heat_df.head()

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)


,latitude,longitude,failure
1,51.472770,-2.633822,1.0
2,51.472770,-2.633822,1.0
3,51.013992,-3.101950,1.0
4,51.080631,-2.612072,1.0
5,51.018373,-3.083549,1.0


In [38]:
# List comprehension to make out list of lists
heat_data = [[row['latitude'],row['longitude'],row['failure']] for index, row in heat_df.iterrows()]

my_map.add_child(plugins.HeatMap(heat_data, min_opacity = 0.2))
my_map.save('failure_map_v1.html')

In [39]:
heat_df

,latitude,longitude,failure
1,51.472770,-2.633822,1.0
2,51.472770,-2.633822,1.0
3,51.013992,-3.101950,1.0
4,51.080631,-2.612072,1.0
5,51.018373,-3.083549,1.0
6,51.018373,-3.083549,1.0
7,51.018373,-3.083549,1.0
8,51.018373,-3.083549,1.0
9,51.410556,-2.491710,1.0
10,50.951161,-2.611821,1.0


In [44]:
my_map2 = folium.Map(location=[51.5074, 0.1278],
                    tiles = 'cartodbpositron',
                    zoom_start = 7) # for UK 

df['latitude'] = df['latitude'].astype(float)
df['longitude'] = df['longitude'].astype(float)
df['success'] = df.loc[:,'outcome'].isin(['Suspect arrested',
                                          'Arrest',
                                          'Offender given drugs possession warning',
                                          'Khat or Cannabis warning',
                                          'Local resolution',
                                          'Article found - Detailed outcome unavailable',
                                          'Summons / charged by post',
                                          'Penalty Notice for Disorder',
                                          'Offender given penalty notice',
                                          'Caution (simple or conditional)',
                                          'Offender cautioned',
                                          'Suspected psychoactive substances seized - No further action']).astype(float)

heat_df = df[['latitude', 'longitude', 'success']]
heat_df.replace(0.0, np.nan, inplace=True)

heat_df = heat_df.dropna(axis=0, subset=['latitude','longitude', 'success'])

heat_data = [[row['latitude'],row['longitude'],row['success']] for index, row in heat_df.iterrows()]

my_map2.add_child(plugins.HeatMap(heat_data, min_opacity = 0.2))
my_map2.save('succes_map_v1.html')

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)


In [42]:
df['outcome'].value_counts()

A no further action disposal                                    303512
False                                                           270428
Suspect arrested                                                 71366
Arrest                                                           64301
Offender given drugs possession warning                          29523
Community resolution                                             21259
Khat or Cannabis warning                                         14551
Local resolution                                                  9140
Article found - Detailed outcome unavailable                      7744
Summons / charged by post                                         7365
Penalty Notice for Disorder                                       7184
Offender given penalty notice                                     6255
Suspect summonsed to court                                        6141
Caution (simple or conditional)                                   2279
Offend